API Chains

If you are just getting started, and you have relatively simple apis, you should get started with chains. Chains are a sequence of predetermined steps, so they are good to get started with as they give you more control and let you understand what is happening better.

In [32]:
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate
from langchain.llms import OpenAI
import os
from langchain.chains.api import open_meteo_docs

os.environ['OPENAI_API_KEY'] = "............"

llm = OpenAI(temperature=0,model_name="gpt-4")

chain_new = APIChain.from_llm_and_api_docs(llm, open_meteo_docs.OPEN_METEO_DOCS, verbose=True)

chain_new.run('What is the weather like right now in Toronto, Canada in degrees celsius?')



> Entering new APIChain chain...
https://api.open-meteo.com/v1/forecast?latitude=43.70&longitude=-79.42&current_weather=true&temperature_unit=celsius
{"latitude":43.70455,"longitude":-79.4046,"generationtime_ms":0.1690387725830078,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":180.0,"current_weather":{"temperature":16.8,"windspeed":8.7,"winddirection":287.0,"weathercode":3,"is_day":0,"time":"2023-06-06T04:00"}}

> Finished chain.


'The current weather in Toronto, Canada is 16.8 degrees Celsius.'

Agents


Agents are more complex, and involve multiple queries to the LLM to understand what to do. The downside of agents are that you have less control. The upside is that they are more powerful, which allows you to use them on larger and more complex schemas.

In [ ]:
import os, yaml

import requests
import os

# URL of the file to download
url = "https://raw.githubusercontent.com/openai/openai-openapi/master/openapi.yaml"

# Send a HTTP request to the specified URL and save the
# response from server in a response object called r
r = requests.get(url)

# Make sure the request was successful
r.raise_for_status()

# Open the file in write mode and write the contents into the file
with open("openapi.yaml", 'wb') as f:
    f.write(r.content)

# Use os.rename() method to rename the file
os.rename("openapi.yaml", "openai_openapi.yaml")


In [ ]:
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec
with open("openai_openapi.yaml") as f:
    raw_openai_api_spec = yaml.load(f, Loader=yaml.Loader)
openai_api_spec = reduce_openapi_spec(raw_openai_api_spec)

In [ ]:
from langchain.llms.openai import OpenAI
from langchain.agents.agent_toolkits.openapi import planner
from langchain.requests import RequestsWrapper

llm = OpenAI(model_name="gpt-4", temperature=0.0)


headers = {
    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"
}
openai_requests_wrapper=RequestsWrapper(headers=headers)


llm = OpenAI(model_name="gpt-4", temperature=0.25)
openai_agent = planner.create_openapi_agent(openai_api_spec, openai_requests_wrapper, llm)
user_query = "generate a short piece of advice"
openai_agent.run(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API call to generate a short piece of advice
Observation: 1. GET /engines to find the appropriate engine for generating text
2. POST /completions with the selected engine and a prompt for generating a short piece of advice
Thought:I have the plan, now I need to execute the API calls.
Action: api_controller
Action Input: 1. GET /engines to find the appropriate engine for generating text
2. POST /completions with the selected engine and a prompt for generating a short piece of advice

> Entering new AgentExecutor chain...
Action: requests_get
Action Input: {"url": "https://api.openai.com/v1/engines", "params": {}, "output_instructions": "Extract the ids of the engines"}
Observation: whisper-1, babbage, davinci, text-davinci-edit-001, babbage-code-search-code, text-similarity-babbage-001, code-davinci-edit-001, text-davinci-001, ada, babbage-code-search-text, babbage-similarity, code-search-

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 97af1194f11e3e41e18429c7f2397787 in your message.).



Observation: The best way to be productive is to develop a good routine and stick to it. Get organized and find a system that works for you. Be disciplined and don't procrastinate. Use a timer to help you stay on track and take
Thought:I am finished executing the plan.
Final Answer: The best way to be productive is to develop a good routine and stick to it. Get organized and find a system that works for you. Be disciplined and don't procrastinate. Use a timer to help you stay on track and take

> Finished chain.

Observation: The best way to be productive is to develop a good routine and stick to it. Get organized and find a system that works for you. Be disciplined and don't procrastinate. Use a timer to help you stay on track and take
Thought:I have executed the plan and generated a short piece of advice.
Final Answer: The best way to be productive is to develop a good routine and stick to it. Get organized and find a system that works for you. Be disciplined and don't procrastinate

"The best way to be productive is to develop a good routine and stick to it. Get organized and find a system that works for you. Be disciplined and don't procrastinate. Use a timer to help you stay on track and take breaks as needed."